In [2]:
from datasets import load_dataset
from tqdm import tqdm
import os
import shutil
import zipfile
import time
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, datasets, transforms
from torchvision.transforms.functional import to_pil_image
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import random
from collections import defaultdict
from PIL import Image

In [5]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Real-world prediction helper
class_names = ["Acne Vulgaris", 
               "Eczema", 
               "Healthy", 
               "Herpes Labialis", 
               "Lichen Planus", 
               "Psoriasis", 
               "Rosacea", 
               "Urticaria"]
num_classes = len(class_names)

def load_model_for_inference(model_path):
    model = models.efficientnet_b0(pretrained=False)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

def predict_real_image(image_path, model, threshold=0.5):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        confidence, preds = torch.max(probabilities, 1)

    if confidence.item() < threshold:
        return "Unknown / No Disease", confidence.item() * 100

    return class_names[preds.item()], confidence.item() * 100



In [7]:
# Example usage:
model_path = "/kaggle/input/model-new-i-think-last/tensorflow2/default/1/best_model.pth"
test_image_path = "/kaggle/input/dermnet/test/Acne and Rosacea Photos/acne-closed-comedo-1.jpg"
model = load_model_for_inference(model_path)
pred_class, conf = predict_real_image(test_image_path, model)
print(f"Prediction: {pred_class}\nConfidence: {conf:.2f}%")

Prediction: Acne Vulgaris
Confidence: 72.00%


<ipython-input-5-60de92214b56>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))
